DATA PART

In [ ]:
import numpy as np
import julian
from datetime import datetime
import pandas as pd
import xlwt

def es_wexler(T, P):
    if T >= 273.15:
        
        # Saturation Vapor Pressure over Water
        g0 =-2.8365744*10**3
        g1 =-6.028076559*10**3
        g2 = 1.954263612*10**1
        g3 =-2.737830188*10**-2
        g4 = 1.6261698*10**-5
        g5 = 7.0229056*10**-10
        g6 =-1.8680009*10**-13
        g7 = 2.7150305
        es = 0.01 * np.exp(g0*T**-2 + g1*T**-1 + g2 + g3*T + g4*T**2 + g5*T**3 + g6*T**4 + g7*np.log(T))

        # Enhancement Factor coefficients for Water 0 to 100°C
        A0 =-1.6302041*10**-1
        A1 = 1.8071570*10**-3
        A2 =-6.7703064*10**-6
        A3 = 8.5813609*10**-9
        B0 =-5.9890467*10**1
        B1 = 3.4378043*10**-1
        B2 =-7.7326396*10**-4
        B3 = 6.3405286*10**-7
    else:
        
        # Saturation Vapor Pressure over Ice
        k0 =-5.8666426*10**3
        k1 = 2.232870244*10**1
        k2 = 1.39387003*10**-2
        k3 =-3.4262402*10**-5
        k4 = 2.7040955*10**-8
        k5 = 6.7063522*10**-1
        es = 0.01 * np.exp(k0*T**-1 + k1 + k2*T + k3*T**2 + k4*T**3 + k5*np.log(T))

        # Enhancement Factor coefficients for Ice –100 to 0°C
        A0 =-6.0190570*10**-2
        A1 = 7.3984060*10**-4
        A2 =-3.0897838*10**-6
        A3 = 4.3669918*10**-9
        B0 =-9.4868712*10**1
        B1 = 7.2392075*10**-1
        B2 =-2.1963437*10**-3
        B3 = 2.4668279*10**-6

       # Enhancement Factor
    alpha = A0 + A1*T + A2*T**2 + A3*T**3
    beta = np.exp(B0 + B1*T + B2*T**2 + B3*T**3)
    f = np.exp( alpha*(1-es/P) + beta*(P/es-1) )
    return es * f

In [ ]:
def write_field_xls(path, sheet_name, value):
    index = len(value)
    workbook = xlwt.Workbook()
    sheet = workbook.add_sheet(sheet_name)
    for k in range(len(idx_R)):
        sheet.write(0, k, idx_R[k])
    for i in range(0, index):
        for j in range(0, len(value[i])):
            sheet.write(i+1, j, value[i][j])
    workbook.save(path)

def hgpt2(dt, x0, y0, z0, z0_type):

    # Grid files location
    coeffiles='' # put '/' or '\' at the end

    # Constants
    row = 721
    col = 1440
    p1  = 365.250
    p2  = 182.625
    p3  = 91.3125

    # Geographic coordinates ( equal to ERA5 )
    lon = np.linspace(-180, 179.75, col)
    lat = np.linspace(-90, 90, row)

    # Modified Julian date
    if np.size(dt) == 6:
        # Input: Gregorian calendar
        mjd = julian.to_jd(datetime(np.int(dt[0]),np.int(dt[1]),np.int(dt[2]),np.int(dt[3]),np.int(dt[4]),np.int(dt[5])), fmt='mjd')
        hour = np.int(dt[3])
    elif np.size(dt) == 1:
        # Input: Modified Julian date
        gre = julian.from_jd(dt, fmt='mjd')
        mjd = dt
        hour = np.int(np.around(gre.hour))
    else:
        raise NameError('Use 1) Modified Julian Date (MJD) or 2) Gregorian date (y,m,d,HH,MM,SS).')

    # Finding indexes for bilinear interpolation
    # x-location
    indx = np.argsort( np.sqrt( (lon-x0)**2 ) )
    ix1 = indx[ 0 ]; ix2 = indx[ 1 ]
    x1 = lon[ ix1 ]; x2 = lon[ ix2 ]
    x = [ix1, ix1, ix2, ix2]
    # y-location
    indy = np.argsort( np.sqrt( (lat-y0)**2 ) )
    jy1 = indy[ 0 ]; jy2 = indy[ 1 ]
    y1 = lat[ jy1 ]; y2 = lat[ jy2 ]
    y = [jy1, jy2, jy1, jy2]
    # xy-distances (weights)
    dx1y1= 1/np.sqrt( (x1 - x0)**2 + (y1 - y0)**2 )
    dx1y2= 1/np.sqrt( (x1 - x0)**2 + (y2 - y0)**2 )
    dx2y1= 1/np.sqrt( (x2 - x0)**2 + (y1 - y0)**2 )
    dx2y2= 1/np.sqrt( (x2 - x0)**2 + (y2 - y0)**2 )
    dxy = np.array([dx1y1, dx1y2, dx2y1, dx2y2], dtype=np.float64)
    if np.any(np.isinf(dxy)) == True:
       # Exact point grid
       dxy = np.array([1,0,0,0], dtype=np.float64)

    # ******************************************************************
    # Open and read the surface air temperature coefficients file
    # ******************************************************************
    fid = open(coeffiles+'temp_grid.bin', 'rb')
    fid.seek((row*col*26)*hour, 0)
    a0 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    b0 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    a1 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f1 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    a2 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f2 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    a3 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f3 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    fid.close()

    # Surface air temperature model
    fun_t = lambda a0, b0, a1, f1, a2, f2, a3, f3: a0 + b0*(mjd - 51178) + a1*np.cos(2*np.pi*(mjd - 51178)/p1+f1) + a2*np.cos(2*np.pi*(mjd - 51178)/p2+f2) + a3*np.cos(2*np.pi*(mjd - 51178)/p3+f3)

    # Applying the bilinear interpolation
    tij = np.array([0,0,0,0], dtype=np.float64)
    for j in range(0, len(x)):
        tij[j] = fun_t(a0[y[j],x[j]], b0[y[j],x[j]], a1[y[j],x[j]], f1[y[j],x[j]], a2[y[j],x[j]], f2[y[j],x[j]], a3[y[j],x[j]], f3[y[j],x[j]])
    T = np.sum(tij*dxy)/np.sum(dxy)

    # ******************************************************************
    # Open and read the surface pressure coefficients file
    # ******************************************************************
    fid = open(coeffiles+'press_grid.bin', 'rb')
    fid.seek((row*col*20)*hour, 0)
    a0 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    b0 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    a1 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f1 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    a2 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f2 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    fid.close()

    # Surface pressure model
    fun_p = lambda a0, b0, a1, f1, a2, f2: a0 + b0*(mjd - 51178) + a1*np.cos(2*np.pi*(mjd - 51178)/p1+f1) + \
    a2*np.cos(2*np.pi*(mjd - 51178)/p2+f2)

    # Applying the bilinear interpolation
    pij = np.array([0,0,0,0], dtype=np.float64)
    for j in range(0, len(x)):
        pij[j] = fun_p(a0[y[j],x[j]], b0[y[j],x[j]], a1[y[j],x[j]], f1[y[j],x[j]], a2[y[j],x[j]], f2[y[j],x[j]])
    P = np.sum(pij*dxy)/np.sum(dxy)

    # ******************************************************************
    # Open and read the surface relative humidity coefficients file
    # ******************************************************************
    fid = open(coeffiles+'rh_grid.bin', 'rb')
    fid.seek((row*col*22)*hour, 0)
    a0 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    a1 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f1 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    a2 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f2 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    a3 = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    f3 = np.fromfile(fid, dtype=np.int16, count=row*col).reshape((row, col), order='F')/10000.0
    fid.close()

    # Surface relative humidity model
    fun_rh = lambda a0, a1, f1, a2, f2, a3, f3: a0 + a1*np.cos(2*np.pi*(mjd - 51178)/p1+f1) + \
    a2*np.cos(2*np.pi*(mjd - 51178)/p2+f2) + a3*np.cos(2*np.pi*(mjd - 51178)/p3+f3)

    # Applying the bilinear interpolation
    rhij = np.array([0,0,0,0], dtype=np.float64)
    for j in range(0, len(x)):
        rhij[j] = fun_rh(a0[y[j],x[j]], a1[y[j],x[j]], f1[y[j],x[j]],a2[y[j],x[j]], f2[y[j],x[j]], a3[y[j],x[j]], f3[y[j],x[j]])
    RH = np.sum(rhij*dxy)/np.sum(dxy)

    # ************************************************************
    # Open and read the Tm coefficients and undulation file
    # ************************************************************
    fid = open(coeffiles+'tm_grid.bin', 'rb')
    a = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    b = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    orography = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    undu = np.fromfile(fid, dtype=np.float32, count=row*col).reshape((row, col), order='F')
    fid.close()

    # Applying the bilinear interpolation
    aij = np.array([0,0,0,0], dtype=np.float64)
    bij = np.copy(aij)
    hij = np.copy(aij)
    nij = np.copy(aij)
    for j in range(0, len(x)):
        aij[j] = a[y[j],x[j]]
        bij[j] = b[y[j],x[j]]
        hij[j] = orography[y[j],x[j]]
        nij[j] = undu[y[j],x[j]]
    a = np.sum(aij*dxy)/np.sum(dxy)
    b = np.sum(bij*dxy)/np.sum(dxy)
    geo_height = np.sum(hij*dxy)/np.sum(dxy)
    N = np.sum(nij*dxy)/np.sum(dxy)

    # Zenith hydrostatic delay (ZHD), Saastamoinen model
    if z0_type=='orth':
        H_orth = z0
    elif z0_type=='elli':
        H_orth = z0 - N
    else:
        raise NameError('Use 1) <<orth>> for Orthometric height or 2) <<elli>> for Ellipsoidal height (in m).')

    # Correction to P, T, and RH (see Guochanf Xu, GPS Theory, Algorithms and Applications, 3nd Edition, page 82)
    dh = H_orth - geo_height

    # Temperature lapse rate by latitude
    # rate = (6.25 - 2*sin( np.deg2rad(y0) )^4)/1000
    rate = (10.3 + 0.03182*(T-273.15) - 0.00436*P)/1000

    # Altimetric corrections
    P = (P*100 * ( 1 - (rate*dh)/T )**5.6004)/100
    T = T - rate * ( dh )

    # Ensure that we eliminate any noise outside this range
    # Gamit has an unidentified error when assuming rh = 100% (met-files)
    if RH >= 100:
        RH = 99.9
    if RH < 0:
        RH = 0

    # Call external function
    es = es_wexler(T, P)
    e = es * (RH/100)

    # Weight mean temperature, Tm
    Tm = a + b*T

    # ZHD using the Saastamoinen Model (see Saastamoinen, 1973)
    ZHD = (0.0022768 * P)/(1 - 0.0026*np.cos(2*np.deg2rad(y0))-0.00000028*H_orth)

    # ZWD using the Saastamoinen Tropospheric Model (see Saastamoinen, 1972, 1973)
    ZWD = 0.002277 * ((1255/T + 0.05) * e)

    # PWV
    # Dimensionless constant
    const = 10**8 / (461525 * (22.9744 + 375463/Tm))
    PWV = ZWD * const

    return ZHD+ZWD


data=pd.read_excel(data_path,header=None)
write_field_xls(outpath,station_name,Last) 

In [ ]:
import numpy as np
from PyEMD import EEMD, EMD, Visualisation,CEEMDAN
import pylab as plt
from datetime import datetime, timezone, timedelta
import pandas as pd
import os

for csv in filelist:
    path=Path
    df=pd.read_csv(path)
    beita=(df['ztd']-df['HGPT2_ZTD']).values
    test=[]
    for k in range(len(df)):
        test.append(k)
    test=np.array(test)
    IMF = CEEMDAN().ceemdan(beita,test,max_imf=10)
    print(len(IMF))
    new_df=df[df_index]
    Beta=Beta_index
    for k,imf in enumerate(IMF[0:10]):
        new_df.insert(new_df.shape[1],Beta[k],imf)
    new_df.insert(new_df.shape[1],'Beta',beita)
    new_df.to_csv(outpath,index=False)

MODEL PART

In [ ]:
import os 
import pandas as pd
from datetime import datetime
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv1D, Flatten, LeakyReLU, Dropout, Input, BatchNormalization,Attention, Input, Multiply,Activation,Reshape
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.metrics import mean_squared_error, explained_variance_score
from keras_adabound import AdaBound
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
import sklearn.preprocessing
import tensorflow.keras.backend as K
from keras.layers import LSTM, Bidirectional, Dense
import warnings
import keras
from keras.layers import LSTM, Bidirectional, Dense

class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.query_transform = Dense(input_shape[0][-1], activation='tanh')
        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        query, value = inputs
        query_transformed = self.query_transform(query)
        attention_scores = tf.keras.layers.Dot(axes=-1)([query_transformed, value])
        attention_scores = tf.keras.layers.Softmax()(attention_scores)
        context_vector = tf.keras.layers.Dot(axes=1)([attention_scores, value])
        return context_vector


def transformer_encoder(inputs, num_layers, d_model, num_heads, dff, training=False):
    # Embedding layer
    x = Dense(d_model, input_shape=inputs.shape[1:])(inputs)

    # MultiHeadAttention layers
    for _ in range(num_layers):
        attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=d_model)(x, x, x, training=training)
        x = tf.keras.layers.add([x, attention_output])  # residual connection
        x = Dense(dff, activation='relu')(x)
        x = Dense(d_model)(x)
        x = Dropout(0.1)(x, training=training)

    return x

def MML(y_true, y_pred, Beta_level):
    # print(y_true.shape)
    # print(y_pred.shape)
    # loss1 = tf.sqrt(tf.reduce_mean(tf.square(y_true[:, Beta_level] - y_pred[:, Beta_level]))) 
    loss1 = tf.reduce_mean(tf.square(y_true[:, Beta_level] - y_pred[:, Beta_level]))
    excluded_dims = [Beta_level, 10]  

    mask = K.ones_like(y_pred)
    for dim in excluded_dims:
        mask = K.concatenate([mask[:, :dim], K.zeros_like(mask[:, dim:dim+1]), mask[:, dim+1:]], axis=1)

    masked_y_pred = y_pred * mask

    Get_sum = K.sum(masked_y_pred, axis=-1, keepdims=True)
    Beta=y_true[:, 10]
    loss2=tf.reduce_mean(tf.square(Beta - Get_sum- y_pred[:, Beta_level]))
    return loss1+loss2

In [ ]:
def GetMyloss(y_true, y_pred, Beta_level):
    # print(y_true.shape)
    # print(y_pred.shape)
    # loss1 = tf.sqrt(tf.reduce_mean(tf.square(y_true[:, Beta_level] - y_pred[:, Beta_level]))) 
    loss1 = tf.reduce_mean(tf.square(y_true[:, Beta_level] - y_pred[:, Beta_level]))
    excluded_dims = [Beta_level, 10]  


    mask = K.ones_like(y_pred)
    for dim in excluded_dims:
        mask = K.concatenate([mask[:, :dim], K.zeros_like(mask[:, dim:dim+1]), mask[:, dim+1:]], axis=1)
 
    masked_y_pred = y_pred * mask

    Get_sum = K.sum(masked_y_pred, axis=-1, keepdims=True)
    Beta=y_true[:, 10]
    loss2=tf.reduce_mean(tf.square(Beta - Get_sum- y_pred[:, Beta_level]))
    return loss1+loss2

In [ ]:
scaler = StandardScaler()

In [ ]:
def TCN_Beta(data, Beta_level):
    X = data[['lat', 'lon', 'hig', 'year', 'mouth', 'day', 'hour', 'P', 'T']]
    y = data[['beita1', 'beita2', 'beita3', 'beita4', 'beita5', 'beita6', 'beita7', 'beita8', 'beita9', 'beita10', 'Beta']]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    X_train = np.expand_dims(X_train, axis=2)
    X_test = np.expand_dims(X_test, axis=2)
    
    kernel_size = 3
    dropout = 0.2
    input_shape = X_train.shape[1:]
    input_layer = Input(shape=input_shape)

    x = transformer_encoder(inputs=input_layer,num_layers=2, d_model=128, num_heads=4, dff=128)
    x = Dense(128, activation="relu")(x)
    # TCN部分
    x = Conv1D(32, kernel_size=3, activation="relu", padding="causal", dilation_rate=1)(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = BatchNormalization()(x)

    x = Conv1D(64, kernel_size=3, padding="causal", activation="relu", dilation_rate=2)(x)
    x = LeakyReLU(alpha=0.01)(x)
    x = BatchNormalization()(x)

    x = Conv1D(128, kernel_size=3, padding="causal", activation="relu", dilation_rate=4)(x)
    x = BatchNormalization()(x)

    
    # 展平和全连接层
    x = Flatten()(x)
    x = Dense(128, activation="relu")(x)
    x = Dense(88, activation="relu")(x)
    x = Dense(44, activation="relu")(x)
    x = Dense(22, activation="relu")(x)
    x = Dropout(dropout)(x)
    output_layer = Dense(11)(x)

    model = Model(inputs=input_layer, outputs=output_layer)
    
    filepath = str(Beta_level) + '\\weights.best.keras'
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]

    model.compile(loss=lambda y_true, y_pred: GetMyloss(y_true, y_pred, Beta_level=Beta_level), optimizer= "adam")

    model.fit(x=X_train, y=y_train, batch_size=500, epochs=300, validation_data=(X_test, y_test), verbose=1, callbacks=callbacks_list)
    
    return model


In [ ]:
data=pd.read_excel('EXAMPLE_COLD.xlsx')
#data=pd.read_excel('EXAMPLE_WARM.xlsx')#切换到自己的cold数据路径下

Generate forecast information

In [ ]:
Beita_=[]
for i in range(10):
    X=data[['lat','lon','hig','year','mouth','day','hour','P','T']]
    X_scaled = scaler.fit_transform(X)
    model_path=str(i+1)+'\\weights.best.keras'
    model=load_model(model_path,custom_objects={'GetMyloss': GetMyloss,'MultiHeadAttention':MultiHeadAttention}, safe_mode=False)
    pred=model.predict(X_scaled)
    Beita_.append(pred)

In [ ]:
Beita_pred=[]
for k in range(len(Beita_)):
    for i in range(len(Beita_[0])):
        beita_=[]
        beita=Beita_[k][i][0]\
             +Beita_[k][i][1]\
             +Beita_[k][i][2]\
             +Beita_[k][i][3]\
             +Beita_[k][i][4]\
             +Beita_[k][i][5]\
             +Beita_[k][i][6]\
             +Beita_[k][i][7]\
             +Beita_[k][i][8]\
             +Beita_[k][i][9]
        beita_.append(beita)
        Beita_pred.append(beita_)

In [ ]:
ztd_pred_1= [data['HGPT2_ZTD'].values.tolist()[i] + Beita_pred[0][i] for i in range(len(data))]
ztd_pred_2= [data['HGPT2_ZTD'].values.tolist()[i] + Beita_pred[1][i] for i in range(len(data))]
ztd_pred_3= [data['HGPT2_ZTD'].values.tolist()[i] + Beita_pred[2][i] for i in range(len(data))]
ztd_pred_4= [data['HGPT2_ZTD'].values.tolist()[i] + Beita_pred[3][i] for i in range(len(data))]
ztd_pred_5= [data['HGPT2_ZTD'].values.tolist()[i] + Beita_pred[4][i] for i in range(len(data))]
ztd_pred_6= [data['HGPT2_ZTD'].values.tolist()[i] + Beita_pred[5][i] for i in range(len(data))]
ztd_pred_7= [data['HGPT2_ZTD'].values.tolist()[i] + Beita_pred[6][i] for i in range(len(data))]
ztd_pred_8= [data['HGPT2_ZTD'].values.tolist()[i] + Beita_pred[7][i] for i in range(len(data))]
ztd_pred_9= [data['HGPT2_ZTD'].values.tolist()[i] + Beita_pred[8][i] for i in range(len(data))]
ztd_pred_10= [data['HGPT2_ZTD'].values.tolist()[i] + Beita_pred[9][i] for i in range(len(data))]

In [ ]:
rmse_1=math.sqrt(mean_squared_error(data['ztd'],ztd_pred_1))
rmse_2=math.sqrt(mean_squared_error(data['ztd'],ztd_pred_2))
rmse_3=math.sqrt(mean_squared_error(data['ztd'],ztd_pred_3))
rmse_4=math.sqrt(mean_squared_error(data['ztd'],ztd_pred_4))
rmse_5=math.sqrt(mean_squared_error(data['ztd'],ztd_pred_5))
rmse_6=math.sqrt(mean_squared_error(data['ztd'],ztd_pred_6))
rmse_7=math.sqrt(mean_squared_error(data['ztd'],ztd_pred_7))
rmse_8=math.sqrt(mean_squared_error(data['ztd'],ztd_pred_8))
rmse_9=math.sqrt(mean_squared_error(data['ztd'],ztd_pred_9))
rmse_10=math.sqrt(mean_squared_error(data['ztd'],ztd_pred_10))